In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_lstm_2025_04_09_20_05.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.000676
1,star_graph_n7,0.0,0.000000
2,star_graph_n7,0.0,0.000000
3,star_graph_n7,0.0,0.000000
4,star_graph_n7,0.0,0.008621


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.000676,0.000676,4.573297e-07,1.0,0.000676
1,star_graph_n7,0.0,0.000000,0.000000,0.000000e+00,1.0,0.000000
2,star_graph_n7,0.0,0.000000,0.000000,0.000000e+00,1.0,0.000000
3,star_graph_n7,0.0,0.000000,0.000000,0.000000e+00,1.0,0.000000
4,star_graph_n7,0.0,0.008621,0.008621,7.432896e-05,1.0,0.008621
...,...,...,...,...,...,...,...
5275,graph_powerlaw_cluster_graph_n7,1.0,1.023407,0.023407,5.478980e-04,0.0,0.023407
5276,graph_powerlaw_cluster_graph_n7,2.0,2.219620,0.219620,4.823273e-02,0.0,0.109810
5277,graph_powerlaw_cluster_graph_n7,1.0,1.017783,0.017783,3.162367e-04,0.0,0.017783
5278,graph_powerlaw_cluster_graph_n7,2.0,1.805249,0.194751,3.792810e-02,0.0,0.097376


In [9]:
df['RE'].sum()/len(df)

0.06345080969105765

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,1736,127.787849,0.073611,7.492294e-02
1,2.0,1602,93.248677,0.058208,2.313698e-02
2,1.0,941,60.969688,0.064792,6.837468e-03
3,4.0,704,47.440453,0.067387,1.035876e-01
4,0.0,225,0.009298,0.000041,3.323835e-07
5,5.0,71,5.519241,0.077736,1.706606e-01
6,6.0,1,0.045069,0.045069,7.312366e-02


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,251.422176,0.064352,0.053116
1,graph_powerlaw_cluster_graph_n7,1350,83.158912,0.061599,0.037585
2,star_graph_n7,23,0.439187,0.019095,0.018160


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3798,251.422176,0.066199,0.054640
1,graph_powerlaw_cluster_graph_n7,1254,83.158912,0.066315,0.040462
2,star_graph_n7,3,0.429889,0.143296,0.139199


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,427,28.834230,0.067527,0.030453
1,graph_powerlaw_cluster_graph_n7,1.0,346,21.821817,0.063069,0.006818
2,graph_powerlaw_cluster_graph_n7,3.0,338,24.415623,0.072236,0.073344
3,graph_powerlaw_cluster_graph_n7,4.0,125,6.972401,0.055779,0.069053
4,graph_powerlaw_cluster_graph_n7,0.0,96,0.000000,0.000000,0.000000
5,graph_powerlaw_cluster_graph_n7,5.0,17,1.069772,0.062928,0.110702
6,graph_powerlaw_cluster_graph_n7,6.0,1,0.045069,0.045069,0.073124
7,graph_random_regular_graph_n7_d4,3.0,1398,103.372226,0.073943,0.075305
8,graph_random_regular_graph_n7_d4,2.0,1173,64.069226,0.054620,0.020310
9,graph_random_regular_graph_n7_d4,1.0,595,39.147871,0.065795,0.006849
